In [ ]:
!pip install -U bitsandbytes accelerate transformers datasets
!apt-get update -qq
!apt-get install fonts-nanum -qq

In [ ]:
# 초기 설정 및 라이브러리 설치
import os
import json
from google.colab import userdata, drive
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import huggingface_hub
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    recall_score
)
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
fm._load_fontmanager(try_read_cache=False)

# 데이터 로드
drive.mount('/content/drive')
df_NLP = pd.read_csv('/content/drive/MyDrive/브라질 이커머스/project_olist_ecommerce/project_dataset/olist_reviews_clean.csv')
df_NLP.info()
n_categories = df_NLP['product_category_name'].nunique()
print(n_categories)

# 클래스 분포
class_dist = df_NLP['review_score'].value_counts().sort_index()
print(class_dist)
print(f"비율:{(class_dist / len(df_NLP) * 100).round(2)}")

# 불균형 체크
max_ratio = class_dist.max() / class_dist.min()
print(f"불균형 비율: {max_ratio:.2f}배")
if max_ratio > 3: #삭제
    print("클래스 불균형 → class_weight 필요")

# 리뷰별 길이 계산
review_lengths = df_NLP['review_comment_message'].str.len()
print(f"최대 길이: {review_lengths.max()}")

# 레이블 생성
df_NLP['sentiment'] = df_NLP['review_score'] - 1  # 1→0, 2→1, 3→2, 4→3, 5→4

print("레이블 매핑 확인")
mapping_check = df_NLP.groupby('review_score')['sentiment'].unique()
for score, sent in mapping_check.items():
    print(f"review_score {score} → sentiment {sent[0]}")

# Phase 1-2: Train/Val/Test 분할 (7:1.5:1.5)
train_df, temp_df = train_test_split(
    df_NLP,
    test_size=0.3,
    stratify=df_NLP['sentiment'],
    random_state=42
)

# Temp: Val(15%) / Test(15%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['sentiment'],
    random_state=42
)

print(f"Train: {len(train_df):,}개 ({len(train_df)/len(df_NLP)*100:.1f}%)")
print(f"Val:   {len(val_df):,}개 ({len(val_df)/len(df_NLP)*100:.1f}%)")
print(f"Test:  {len(test_df):,}개 ({len(test_df)/len(df_NLP)*100:.1f}%)")

# 클래스 분포 확인
print("Train 클래스 분포:")
print(train_df['sentiment'].value_counts().sort_index())

# 토큰 길이 확인
# Hugging Face 로그인
huggingface_hub.login(token=userdata.get('NLP_TOKEN'))
MODEL_ID = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# 샘플 1000개로 토큰 길이 측정
sample_texts = df_NLP['review_comment_message'].sample(1000, random_state=42)
token_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in sample_texts]

print(f"토큰 길이 통계:")
print(f"평균: {sum(token_lengths)/len(token_lengths):.1f}")
print(f"95%: {sorted(token_lengths)[int(len(token_lengths)*0.95)]}")
print(f"99%: {sorted(token_lengths)[int(len(token_lengths)*0.99)]}")

# max_length 권장
max_length = sorted(token_lengths)[int(len(token_lengths)*0.95)]
if max_length <= 128:
    print(f"max_length: 128")
else:
    print(f"max_length: {max_length}")

# 데이터 저장
BASE_PATH = '/content/drive/MyDrive/브라질 이커머스/NLP'
SAVE_PATH = f'{BASE_PATH}/processed'
os.makedirs(SAVE_PATH, exist_ok=True)

train_df.to_csv(f'{SAVE_PATH}/train.csv', index=False)
val_df.to_csv(f'{SAVE_PATH}/val.csv', index=False)
test_df.to_csv(f'{SAVE_PATH}/test.csv', index=False)


In [ ]:
import re
# 방법: 이모지 확인

# 이모지 패턴 (Unicode 범위)
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # 감정 표현 (😀-🙏)
    "\U0001F300-\U0001F5FF"  # 기호 & 픽토그램 (🌀-🗿)
    "\U0001F680-\U0001F6FF"  # 교통 & 지도 (🚀-🛿)
    "\U0001F1E0-\U0001F1FF"  # 국기 (🇦-🇿)
    "\U00002702-\U000027B0"  # Dingbats (✂-➰)
    "\U000024C2-\U0001F251"  # 기타
    "]+",
    flags=re.UNICODE
)

# 전체 데이터 확인
has_emoji = test_df['review_comment_message'].apply(
    lambda x: bool(emoji_pattern.search(str(x)))
)

print("이모지 포함 여부")
print(f"총 리뷰:        {len(test_df):,}개")
print(f"이모지 포함:    {has_emoji.sum():,}개 ({has_emoji.mean()*100:.2f}%)")
print(f"이모지 없음:    {(~has_emoji).sum():,}개 ({(~has_emoji).mean()*100:.2f}%)")

# 방법 3: 샘플로 확인 (빠른 체크)
import random

# 이모지 포함 리뷰만 추출
emoji_reviews = test_df[test_df['review_comment_message'].apply(
    lambda x: bool(emoji_pattern.search(str(x)))
)]

if len(emoji_reviews) > 0:
    print("="*60)
    print(f"이모지 포함 리뷰 샘플 ({len(emoji_reviews)}개 중 10개)")
    print("="*60)

    samples = emoji_reviews.sample(min(10, len(emoji_reviews)), random_state=42)

    for idx, (_, row) in enumerate(samples.iterrows(), 1):
        print(f"\n{idx}. 감성: {row['sentiment']} | 예측: {row.get('predicted', 'N/A')}")
        print(f"   {row['review_comment_message'][:150]}...")
else:
    print("✅ 이모지 포함 리뷰 없음")

In [ ]:
# 2-1: Test Set 로드
SAVE_PATH = '/content/drive/MyDrive/브라질 이커머스/NLP/processed'
test_df = pd.read_csv(f'{SAVE_PATH}/test.csv')

print(f"Test set: {len(test_df):,}개")
print(f"클래스 분포:\n{test_df['sentiment'].value_counts().sort_index()}")

device = torch.device('cuda')

# 2-2: 모델 로드 및 추론 준비
# Hugging Face 로그인
huggingface_hub.login(token=userdata.get('NLP_TOKEN'))
MODEL_ID = "tabularisai/multilingual-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
model.to(device)
model.eval()

# 2-3: Test Set 추론 (배치 처리)
def predict_batch(texts, batch_size=32):
    predictions = []

    for i in tqdm(range(0, len(texts), batch_size), desc="추론 중"):
        batch_texts = texts[i:i+batch_size]

        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            predictions.extend(preds)

    return np.array(predictions)

# 추론 실행
test_texts = test_df['review_comment_message'].tolist()
y_true = test_df['sentiment'].values
y_pred = predict_batch(test_texts, batch_size=32)

print(f"\n추론 완료: {len(y_pred):,}개")

# 2-4: F1-score 계산
f1_weighted = f1_score(y_true, y_pred, average='weighted')
f1_macro = f1_score(y_true, y_pred, average='macro')
accuracy = (y_true == y_pred).mean()

print("Baseline 성능")
print(f"Accuracy:        {accuracy:.4f}")
print(f"F1 (Weighted):   {f1_weighted:.4f}")
print(f"F1 (Macro):      {f1_macro:.4f}")
print(f"\n목표 F1 (Weighted): {f1_weighted + 0.05:.4f} (+5%)")

# 2-5: 클래스별 성능 확인
print("클래스별 성능")
print(classification_report(
    y_true,
    y_pred,
    target_names=['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive'],
    digits=4
))

# 2-6: Confusion Matrix 시각화
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['0', '1', '2', '3', '4'],
    yticklabels=['0', '1', '2', '3', '4']
)
plt.title('Baseline Confusion Matrix', fontsize=14, weight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()

# 저장
RESULT_PATH = f'{SAVE_PATH}/results'
os.makedirs(RESULT_PATH, exist_ok=True)
plt.savefig(f'{RESULT_PATH}/baseline_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# 2-7: 결과 저장 (JSON)
per_class_f1 = f1_score(y_true, y_pred, average=None)

baseline_results = {
    "model": MODEL_ID,
    "test_samples": len(test_df),
    "accuracy": float(accuracy),
    "f1_weighted": float(f1_weighted),
    "f1_macro": float(f1_macro),
    "target_f1": float(f1_weighted + 0.05),
    "class_distribution": test_df['sentiment'].value_counts().sort_index().to_dict(),
    "per_class_f1": {
        str(i): float(per_class_f1[i])
        for i in range(5)
    }
}

with open(f'{RESULT_PATH}/baseline_metrics.json', 'w') as f:
    json.dump(baseline_results, f, indent=2)

# 요약 출력
print(f"Baseline F1 (Weighted): {f1_weighted:.4f}")
print(f"목표 F1 (LoRA 후):      {f1_weighted + 0.05:.4f}")
print(f"개선 필요량:            +{0.05:.4f} ({0.05/f1_weighted*100:.1f}%)")

test_df.info()

In [ ]:
# ============================================
# Phase 3: LoRA 파인튜닝 파이프라인
# ============================================
!pip install -q peft accelerate

from peft import LoraConfig, get_peft_model, TaskType
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset

# ============================================
# 3-1: Train/Val 데이터 로드
# ============================================
SAVE_PATH = '/content/drive/MyDrive/브라질 이커머스/NLP/processed'
train_df = pd.read_csv(f'{SAVE_PATH}/train.csv')
val_df = pd.read_csv(f'{SAVE_PATH}/val.csv')

print(f"Train: {len(train_df):,}개")
print(f"Val:   {len(val_df):,}개")
print(f"\nTrain 클래스 분포:")
print(train_df['sentiment'].value_counts().sort_index())

# ============================================
# 3-2: Class Weight 계산
# ============================================
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_df['sentiment']),
    y=train_df['sentiment']
)

print(f"\nClass Weights:")
for i, weight in enumerate(class_weights):
    count = (train_df['sentiment'] == i).sum()
    print(f"  클래스 {i}: {weight:.2f} (샘플 {count}개)")

# Tensor로 변환
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to('cuda')

# ============================================
# 3-3: 데이터셋 토큰화
# ============================================
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

# Hugging Face Dataset 형식으로 변환
train_dataset = Dataset.from_dict({
    'text': train_df['review_comment_message'].tolist(),
    'label': train_df['sentiment'].tolist()
})

val_dataset = Dataset.from_dict({
    'text': val_df['review_comment_message'].tolist(),
    'label': val_df['sentiment'].tolist()
})

# 토큰화
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print(f"\n✅ 토큰화 완료")

# ============================================
# 3-4: LoRA Config 설정
# ============================================
# 베이스 모델 로드
MODEL_ID = "tabularisai/multilingual-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],  # DistilBERT attention
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# LoRA 모델 생성
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ============================================
# 3-5: Custom Trainer (Class Weight 적용)
# ============================================
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Class weight 적용한 Loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# ============================================
# 3-6: Metrics 계산 (Macro-F1 + Per-class Recall)
# ============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)

    # Macro F1 (Early stopping 기준)
    macro_f1 = f1_score(labels, preds, average='macro')

    # Weighted F1 (실무 성능)
    weighted_f1 = f1_score(labels, preds, average='weighted')

    # Per-class Recall (클래스 4 모니터링)
    recall_per_class = recall_score(y_true, y_pred, average=None)

    return {
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1,
        'recall_0': recall_per_class[0],
        'recall_1': recall_per_class[1],
        'recall_2': recall_per_class[2],
        'recall_3': recall_per_class[3],
        'recall_4': recall_per_class[4],  # ⭐ 가장 중요
    }

# ============================================
# 3-7: Training Arguments
# ============================================
CHECKPOINT_PATH = f'{SAVE_PATH}/checkpoints'

training_args = TrainingArguments(
    output_dir=CHECKPOINT_PATH,

    # 학습 파라미터
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-4,
    weight_decay=0.01,

    # 평가 & 저장 전략
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,

    # Best model 관리
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",  # Macro-F1 기준
    greater_is_better=True,
    save_total_limit=2,

    # 모니터링
    logging_steps=50,
    logging_dir=f'{CHECKPOINT_PATH}/logs',

    # 기타
    fp16=True,  # 메모리 절약
    report_to="none",
)

# ============================================
# 3-8: Trainer 생성
# ============================================
from transformers import EarlyStoppingCallback

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=3,
        early_stopping_threshold=0.001
    )]
)

In [ ]:
# Phase 4: 모델 학습 및 검증
trainer.train()

# Loss 그래프: 학습 로그 추출
log_history = trainer.state.log_history
train_logs = [x for x in log_history if 'loss' in x and 'eval_loss' not in x]
eval_logs = [x for x in log_history if 'eval_loss' in x]

# 그래프 생성
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss 그래프
axes[0].plot([x['step'] for x in train_logs], [x['loss'] for x in train_logs], label='Train Loss')
axes[0].plot([x['step'] for x in eval_logs], [x['eval_loss'] for x in eval_logs], label='Eval Loss', marker='o')
axes[0].set_xlabel('Steps')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Macro-F1 그래프
axes[1].plot([x['step'] for x in eval_logs], [x['eval_macro_f1'] for x in eval_logs], marker='o', color='green')
axes[1].set_xlabel('Steps')
axes[1].set_ylabel('Macro-F1')
axes[1].set_title('Validation Macro-F1')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULT_PATH}/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# 4-3: Test Set 평가
print("Test Set 최종 평가")

# Test 데이터 로드
test_df = pd.read_csv(f'{SAVE_PATH}/test.csv')
test_dataset = Dataset.from_dict({
    'text': test_df['review_comment_message'].tolist(),
    'label': test_df['sentiment'].tolist()
})
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# 추론
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = test_df['sentiment'].values

# 성능 계산
from sklearn.metrics import recall_score

weighted_f1 = f1_score(y_true, y_pred, average='weighted')
macro_f1 = f1_score(y_true, y_pred, average='macro')
accuracy = (y_true == y_pred).mean()
recall_per_class = recall_score(y_true, y_pred, average=None)

# 결과 출력
print(f"\nAccuracy:        {accuracy:.4f}")
print(f"Weighted F1:     {weighted_f1:.4f}")
print(f"Macro F1:        {macro_f1:.4f}")

print(f"\n클래스별 Recall:")
for i in range(5):
    print(f"  클래스 {i}: {recall_per_class[i]:.4f}")

# 4-4: Baseline 비교
print("Baseline vs LoRA 비교")

baseline_recall_4 = 0.0892  # Phase 2 결과
lora_recall_4 = recall_per_class[4]

print(f"\n⭐ Recall(4) 개선:")
print(f"  Baseline: {baseline_recall_4:.4f}")
print(f"  LoRA:     {lora_recall_4:.4f}")
print(f"  개선량:   +{lora_recall_4 - baseline_recall_4:.4f} ({(lora_recall_4/baseline_recall_4 - 1)*100:.1f}%)")

if lora_recall_4 >= 0.40:
    print(f"\n✅ 목표 달성! (Recall 4 ≥ 0.40)")
else:
    print(f"\n⚠️ 목표 미달 (목표: 0.40, 현재: {lora_recall_4:.4f})")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['0','1','2','3','4'],
            yticklabels=['0','1','2','3','4'])
plt.title('LoRA Fine-tuned Confusion Matrix', fontsize=14, weight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(f'{RESULT_PATH}/lora_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# 결과 저장
lora_results = {
    "accuracy": float(accuracy),
    "weighted_f1": float(weighted_f1),
    "macro_f1": float(macro_f1),
    "recall_per_class": {str(i): float(recall_per_class[i]) for i in range(5)},
    "recall_4_baseline": baseline_recall_4,
    "recall_4_improvement": float(lora_recall_4 - baseline_recall_4)
}

with open(f'{RESULT_PATH}/lora_results.json', 'w') as f:
    json.dump(lora_results, f, indent=2)


# ============================================
# Phase 4: 성능 지표 해석 추가: 중복 확인
# ============================================
print(f"\n클래스별 Recall:")
for i in range(5):
    print(f"  클래스 {i}: {recall_per_class[i]:.4f}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🆕 성능 지표 해석 추가
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*70)
print("📊 성능 지표 해석")
print("="*70)

print(f"\n✅ Weighted F1 높음 ({weighted_f1:.3f})")
print("   → 실사용 성능 우수")
print("   → 다수 클래스(Very Positive) 잘 예측하여 전체 커버리지 ↑")
print(f"   → 고객 만족도 모니터링 정확도: 74.7%")

print(f"\n⚠️  Macro F1 낮음 ({macro_f1:.3f})")
print("   → 소수 클래스(Negative, Neutral, Positive) 여전히 어려움")
print("   → 클래스 간 성능 불균형 존재")
print(f"   → 특히 Neutral(클래스 2) 가장 취약: {recall_per_class[2]:.1%}")

# print(f"\n💡 비즈니스 관점")
# print("   목표: Very Positive 리뷰 자동 감지")
# print("   결과: Weighted F1 개선이 더 중요 ✅")
# print("   근거:")
# print("     - 전체 리뷰의 50.2%가 Very Positive (클래스 4)")
# print("     - 해당 클래스 Recall 8.9% → 74.7% (+65.8%p)")
# print(f"     - 실무 활용 가능 수준 달성")

print(f"\n📈 개선 요약")
print(f"   Baseline Weighted F1: 0.1891")
print(f"   LoRA Weighted F1:     {weighted_f1:.4f}")
print(f"   개선량:               +{weighted_f1 - 0.1891:.4f} ({(weighted_f1/0.1891 - 1)*100:.1f}%)")

In [ ]:
# # 1-1: 오류 분석 - 어떤 리뷰가 틀렸는지 확인
# error_df = test_df.copy()
# error_df['predicted'] = y_pred
# error_df['correct'] = (y_true == y_pred)

# # 클래스 2 오분류 케이스 추출
# class2_errors = error_df[
#     (error_df['sentiment'] == 2) &
#     (error_df['correct'] == False)
# ].sample(20)

# print("Neutral 오분류 샘플:")
# for idx, row in class2_errors.iterrows():
#     print(f"\n실제: {row['sentiment']} | 예측: {row['predicted']}")
#     print(f"리뷰: {row['review_comment_message'][:100]}...")

# # 1-2: 모델 저장 (Hugging Face Hub 업로드)
# MODEL_NAME = "hywan/olist-sentiment-lora"

# # LoRA 어댑터만 저장 (540MB → 3MB)
# model.save_pretrained(f'{SAVE_PATH}/lora_adapter')
# tokenizer.save_pretrained(f'{SAVE_PATH}/lora_adapter')

# # Hub에 업로드
# model.push_to_hub(MODEL_NAME)
# tokenizer.push_to_hub(MODEL_NAME)

# # 비즈니스 임팩트 계산
# baseline_detected = int(5024 * 0.0892)  # 448개
# lora_detected = int(5024 * 0.7470)      # 3,752개

# print(f"✅ 추가 감지: {lora_detected - baseline_detected:,}개 긍정 리뷰")
# print(f"   → 고객 만족도 모니터링 커버리지 {(lora_detected/2522)*100:.1f}%")

In [ ]:
# 5-1: 예측 결과 통합
# ============================================
# 기존 결과 활용
test_result = test_df.copy()
test_result['predicted'] = y_pred
test_result['correct'] = (y_true == y_pred)

print(f"✅ 분석 데이터: {len(test_result):,}개")
print(f"카테고리 수: {test_result['product_category_name'].nunique()}개")

# ============================================
# Phase 5: 부정 키워드 & 카테고리 분석 (간결 버전)
# ============================================
from collections import Counter
import re

# 포르투갈어 핵심 불용어
stopwords = {'de', 'a', 'o', 'que', 'e', 'não', 'para', 'com', 'em', 'um'}

def get_keywords(reviews, n=10):
    """키워드 추출"""
    words = []
    for text in reviews:
        if pd.notna(text):
            words.extend([
                w for w in re.findall(r'\b[a-záàâãéêíóôõúç]+\b', text.lower())
                if w not in stopwords and len(w) > 3
            ])
    return Counter(words).most_common(n)

# 1. 전체 부정 키워드 Top 10
print("1️⃣ 전체 부정 키워드 Top 10")

neg_reviews = test_result[test_result['sentiment'].isin([0, 1])]['review_comment_message']
keywords = get_keywords(neg_reviews, 10)

for i, (word, cnt) in enumerate(keywords, 1):
    print(f"{i:2}. {word:20} {cnt:4}회")

# 2. 부정 비율 높은 카테고리 Top 5
print("\n" + "="*70)
print("2️⃣ 부정 비율 높은 카테고리 Top 5")

cat_neg = test_result.groupby('product_category_name').apply(
    lambda x: pd.Series({
        'total': len(x),
        'neg_cnt': x['sentiment'].isin([0, 1]).sum(),
        'neg_ratio': x['sentiment'].isin([0, 1]).sum() / len(x)
    }) if len(x) >= 10 else None
).dropna().sort_values('neg_ratio', ascending=False).head(5)

for i, (cat, row) in enumerate(cat_neg.iterrows(), 1):
    print(f"{i}. {cat:40} {row['neg_ratio']:.1%} ({int(row['neg_cnt'])}/{int(row['total'])})")

# 3. 각 카테고리별 부정 키워드 Top 3
print("\n" + "="*70)
print("3️⃣ 각 카테고리별 부정 키워드 Top 3")

for i, (cat, _) in enumerate(cat_neg.iterrows(), 1):
    cat_neg_reviews = test_result[
        (test_result['product_category_name'] == cat) &
        (test_result['sentiment'].isin([0, 1]))
    ]['review_comment_message']

    kws = get_keywords(cat_neg_reviews, 3)

    print(f"\n{i}. {cat}")
    for j, (word, cnt) in enumerate(kws, 1):
        print(f"   {j}) {word:15} {cnt:3}회")

In [ ]:
# ============================================
# Phase 5A: 예측 확률 기반 오류 분석
# ============================================
# 확률값 추출
probs = predictions.predictions  # (5024, 5) 형태
test_result['max_prob'] = np.max(probs, axis=1)
test_result['second_prob'] = np.partition(probs, -2, axis=1)[:, -2]
test_result['prob_gap'] = test_result['max_prob'] - test_result['second_prob']

print("✅ 예측 확률 저장 완료")
print(f"평균 확률 차이: {test_result['prob_gap'].mean():.3f}")

# ============================================
# 2. Low Confidence 샘플 분석
# ============================================
print("\n" + "="*70)
print("📊 Confidence 분포 분석")
print("="*70)

# 정답 vs 오답 확률 비교
correct_conf = test_result[test_result['correct']]['max_prob'].mean()
wrong_conf = test_result[~test_result['correct']]['max_prob'].mean()

print(f"\n정답 평균 확률:   {correct_conf:.3f}")
print(f"오답 평균 확률:   {wrong_conf:.3f}")
print(f"차이:            {correct_conf - wrong_conf:.3f}")

# Low confidence 샘플 (확률 < 0.5)
low_conf = test_result[test_result['max_prob'] < 0.5]
print(f"\nLow Confidence (<0.5): {len(low_conf)}개 ({len(low_conf)/len(test_result)*100:.1f}%)")

# 애매한 샘플 (확률 차이 < 0.1)
uncertain = test_result[test_result['prob_gap'] < 0.1]
print(f"애매한 샘플 (<0.1 차이): {len(uncertain)}개 ({len(uncertain)/len(test_result)*100:.1f}%)")

# ============================================
# 3. 애매한 샘플 직접 확인 (10개)
# ============================================
print("\n" + "="*70)
print("🔍 애매한 샘플 직접 확인 (확률 차이 < 0.1)")
print("="*70)

class_names = ['Very Neg', 'Neg', 'Neutral', 'Pos', 'Very Pos']

uncertain_sample = uncertain.sample(min(10, len(uncertain)), random_state=42)

for idx, (_, row) in enumerate(uncertain_sample.iterrows(), 1):
    status = "✅" if row['correct'] else "❌"
    print(f"\n{idx}. {status} | 실제: {class_names[row['sentiment']]} | 예측: {class_names[row['predicted']]}")
    print(f"   확률: {row['max_prob']:.2f} (2등: {row['second_prob']:.2f}, 차이: {row['prob_gap']:.2f})")
    print(f"   리뷰: {row['review_comment_message'][:120]}...")
    print(f"   카테고리: {row['product_category_name']}")

# ============================================
# 4. 자주 틀리는 클래스 조합
# ============================================
print("\n" + "="*70)
print("⚠️ 자주 틀리는 클래스 조합 (Top 10)")
print("="*70)

# 오분류만 추출
misclassified = test_result[~test_result['correct']]

# 클래스 조합별 집계
error_combos = misclassified.groupby(['sentiment', 'predicted']).agg({
    'max_prob': 'mean',  # 평균 확률
    'review_comment_message': 'count'  # 빈도
}).rename(columns={'review_comment_message': 'count'}).reset_index()

error_combos = error_combos.sort_values('count', ascending=False).head(10)

print(f"\n{'실제':<12} {'예측':<12} {'빈도':<8} {'평균 확률':<10}")
print("-" * 45)

for _, row in error_combos.iterrows():
    true_cls = class_names[int(row['sentiment'])]
    pred_cls = class_names[int(row['predicted'])]
    print(f"{true_cls:<12} {pred_cls:<12} {int(row['count']):<8} {row['max_prob']:.3f}")

# ============================================
# 5. 액션 태그 달기
# ============================================
print("\n" + "="*70)
print("🏷️  액션 태그 분류")
print("="*70)

def assign_action_tag(row):
    """오분류 샘플에 액션 태그 부여"""

    # 고확신 오답 → 재라벨링 후보
    if row['max_prob'] > 0.8 and not row['correct']:
        return '재라벨링_후보'

    # 저확신 → 전처리 개선
    elif row['max_prob'] < 0.5:
        return '전처리_개선'

    # 극단 오분류 (0↔4, 1↔4) → 규칙 보완
    elif abs(row['sentiment'] - row['predicted']) >= 3:
        return '규칙_보완_후보'

    # 정답
    elif row['correct']:
        return '정답'

    # 일반 오답
    else:
        return '일반_오류'

# 태그 적용
test_result['action_tag'] = test_result.apply(assign_action_tag, axis=1)

# 태그별 요약
tag_summary = test_result['action_tag'].value_counts()
print("\n액션 태그별 분포:")
for tag, count in tag_summary.items():
    print(f"  {tag:20} {count:5}개 ({count/len(test_result)*100:.1f}%)")

# ============================================
# 6. 액션별 샘플 예시
# ============================================
print("\n" + "="*70)
print("📋 액션별 샘플 예시")
print("="*70)

for tag in ['재라벨링_후보', '전처리_개선', '규칙_보완_후보']:
    tag_samples = test_result[test_result['action_tag'] == tag]

    if len(tag_samples) > 0:
        print(f"\n🔹 {tag} ({len(tag_samples)}개)")

        for idx, (_, row) in enumerate(tag_samples.sample(min(3, len(tag_samples)), random_state=42).iterrows(), 1):
            print(f"\n  {idx}. 실제: {class_names[row['sentiment']]} | 예측: {class_names[row['predicted']]} | 확률: {row['max_prob']:.2f}")
            print(f"     {row['review_comment_message'][:100]}...")

# ============================================
# 7. 인사이트 정리
# ============================================
print("\n" + "="*70)
print("💡 오류 분석 인사이트")
print("="*70)

relabel_cnt = (test_result['action_tag'] == '재라벨링_후보').sum()
preprocess_cnt = (test_result['action_tag'] == '전처리_개선').sum()
rule_cnt = (test_result['action_tag'] == '규칙_보완_후보').sum()

print(f"\n1️⃣ 재라벨링 필요 (고확신 오답)")
print(f"   샘플 수: {relabel_cnt}개")
print(f"   → 데이터 품질 검토 필요")

print(f"\n2️⃣ 전처리 개선 (저확신)")
print(f"   샘플 수: {preprocess_cnt}개")
print(f"   → 짧은 리뷰, 애매한 표현 처리 개선")

print(f"\n3️⃣ 규칙 보완 (극단 오분류)")
print(f"   샘플 수: {rule_cnt}개")
print(f"   → 비꼬는 표현, 혼합 감정 처리 필요")

print(f"\n4️⃣ 하이퍼파라미터 조정 방향")
if preprocess_cnt > relabel_cnt:
    print("   → Dropout 증가 (모델 확신도 조절)")
    print("   → Temperature Scaling 고려")
else:
    print("   → 데이터 증강 우선 (특히 클래스 1, 2, 3)")
    print("   → Class Weight 재조정")

# Top 오류 조합 기반
top_error = error_combos.iloc[0]
print(f"\n5️⃣ 가장 많은 오류")
print(f"   {class_names[int(top_error['sentiment'])]} → {class_names[int(top_error['predicted'])]}: {int(top_error['count'])}건")
print(f"   → 해당 조합 집중 분석 필요")

print("\n" + "="*70)
print("✅ Phase 5A 완료")
print("="*70)

# 결과 저장
analysis_results = {
    "confidence_analysis": {
        "정답_평균확률": float(correct_conf),
        "오답_평균확률": float(wrong_conf),
        "low_confidence_count": int(len(low_conf)),
        "uncertain_count": int(len(uncertain))
    },

    "action_tags": {
        tag: int(count)
        for tag, count in tag_summary.items()
    },

    "top_error_patterns": [
        {
            "실제": class_names[int(row['sentiment'])],
            "예측": class_names[int(row['predicted'])],
            "빈도": int(row['count']),
            "평균_확률": float(row['max_prob'])
        }
        for _, row in error_combos.head(5).iterrows()
    ]
}

with open(f'{RESULT_PATH}/error_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(analysis_results, f, indent=2, ensure_ascii=False)

print(f"\n✅ 분석 결과 저장: {RESULT_PATH}/error_analysis.json")

In [ ]:
# ============================================
# 오류 분석 샘플 직접 검토
# ============================================

class_names = ['Very Neg', 'Neg', 'Neutral', 'Pos', 'Very Pos']

def show_samples(df, title, n=10):
    """샘플 n개 출력"""
    print(f"\n{'='*70}")
    print(f"🔍 {title} (총 {len(df)}개 중 {min(n, len(df))}개)")
    print('='*70)

    samples = df.sample(min(n, len(df)), random_state=42)

    for idx, (_, row) in enumerate(samples.iterrows(), 1):
        status = "✅" if row['correct'] else "❌"
        print(f"\n[{idx}] {status} | 확률: {row['max_prob']:.2f} | 차이: {row['prob_gap']:.2f}")
        print(f"    실제: {class_names[row['sentiment']]:12} → 예측: {class_names[row['predicted']]}")
        print(f"    📝 {row['review_comment_message'][:150]}")
        print(f"    📦 {row['product_category_name']}")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1️⃣ 재라벨링 후보 (고확신 오답)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
relabel_samples = test_result[test_result['action_tag'] == '재라벨링_후보']
show_samples(relabel_samples, '재라벨링 후보 - 데이터 품질 검토 필요')


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2️⃣ 전처리 개선 (저확신)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
preprocess_samples = test_result[test_result['action_tag'] == '전처리_개선']
show_samples(preprocess_samples, '전처리 개선 - 짧은 리뷰/애매한 표현')


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3️⃣ 규칙 보완 (극단 오분류)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
rule_samples = test_result[test_result['action_tag'] == '규칙_보완_후보']
show_samples(rule_samples, '규칙 보완 - 비꼬는 표현/혼합 감정', n=min(10, len(rule_samples)))


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 4️⃣ 가장 많은 오류: Very Pos → Pos
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
verypos_to_pos = test_result[
    (test_result['sentiment'] == 4) &
    (test_result['predicted'] == 3)
]
show_samples(verypos_to_pos, 'Very Pos → Pos 오류 (566건)')


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 5️⃣ 추가: Very Neg → Neg (2번째로 많은 오류)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
veryneg_to_neg = test_result[
    (test_result['sentiment'] == 0) &
    (test_result['predicted'] == 1)
]
show_samples(veryneg_to_neg, 'Very Neg → Neg 오류 (262건)')


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📊 요약 테이블
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print(f"\n{'='*70}")
print("📊 검토 대상 요약")
print('='*70)

summary_data = {
    '카테고리': [
        '재라벨링 후보',
        '전처리 개선',
        '규칙 보완',
        'Very Pos → Pos',
        'Very Neg → Neg'
    ],
    '샘플 수': [
        len(relabel_samples),
        len(preprocess_samples),
        len(rule_samples),
        len(verypos_to_pos),
        len(veryneg_to_neg)
    ],
    '우선순위': [
        '⭐⭐⭐ 최우선',
        '⭐⭐ 중요',
        '⭐ 낮음',
        '⭐⭐⭐ 최우선',
        '⭐⭐ 중요'
    ]
}

for i in range(len(summary_data['카테고리'])):
    print(f"{summary_data['카테고리'][i]:20} {summary_data['샘플 수'][i]:6}개   {summary_data['우선순위'][i]}")

print(f"\n💡 검토 순서: Very Pos↔Pos → 재라벨링 후보 → 전처리 개선")